# Pedía a Gritos un Título

In [1]:
from finrisk import QC_Financial_3 as Qcf
from typing import List, Tuple, Callable
from scipy.interpolate import interp1d

import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

#from modules
import auxiliary as aux

from numpy import random as rnd
import plotly.express as px
import pandas as pd
from scipy.stats import norm
import numpy as np
import math

In [2]:
curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')
curva.head()#.style.format(frmt)
curva['t'] = curva['plazo'] / 365.0
curva['log_df'] = np.log(curva['df'])
curva.head()

,plazo,tasa,df,t,log_df
0,1,0.000811,0.999998,0.002740,-0.000002
1,7,0.000841,0.999984,0.019178,-0.000016
2,14,0.000780,0.999970,0.038356,-0.000030
3,21,0.000774,0.999955,0.057534,-0.000045
4,33,0.000781,0.999929,0.090411,-0.000071


In [3]:
zcurva = interp1d(curva['t'],
                  curva['tasa'],
                  kind='cubic',
                  fill_value="extrapolate")
steps_per_year = 365
years = 20
dt = 1 / steps_per_year
result = []
for i in range(0, steps_per_year * years):
    result.append((i * dt, zcurva(i * dt)))
df_result = pd.DataFrame(result, columns=['plazo', 'tasa'])

**AD:** primeras celdas sin comentarios.

In [4]:
# FUNCIONES CON ELEMENTOS PARA FÓRMULA CERRADA Y SIMULACIONES DE MONTECARLO

def zrate(t):
    return zcurva(t)


def dzrate(t):
    delta = .0001
    return (zrate(t + delta) - zrate(t)) / delta


def d2zrate(t):
    delta = .0001
    return (dzrate(t + delta) - dzrate(t)) / delta


def fwd(t):
    return zrate(t) + dzrate(t)*t


def dfwd(t):
    return 2*dzrate(t) + d2zrate(t)*t


def bhw(gamma, t, T):

    aux = 1 - math.exp(-gamma*(T-t))
    B = aux/gamma
    return B


def ahw(gamma, sigma, t, T):
    b = bhw(gamma, t, T)
    dfT = math.exp(-zrate(T)*T)
    dft = math.exp(-zrate(t)*t)
    c1 = math.log(dfT/dft)
    c2 = b*fwd(t)
    c3 = (sigma**2) / (4*gamma)*(b**2)*(1-math.exp(-2*gamma*t))
    A = c1 + c2 - c3
    return A


# No usaremos la medida forward, por lo que el theta no presenta el uso del valor "B"
def theta(t, gamma, sigma):
    aux = (sigma ** 2) / (2.0 * gamma) * (1 - math.exp(-2.0 * gamma * t))
    return dfwd(t) + gamma * fwd(t) + aux


def Sz(gamma, sigma, TO, TB):

    B = bhw(gamma, TO, TB)
    z = 1-math.exp(-2*gamma*TO)
    x = (sigma**2)/(2*gamma)
    w = B*math.sqrt(x*z)
    return w


def zero_hw(r, gamma, sigma, t, T):
    a = ahw(gamma, sigma, t, T)
    b = bhw(gamma, t, T)
    u = math.exp(a-b*r)
    return u


def d1(r, gamma, sigma, t, TO, TB, Strike):
    Z_TB = zero_hw(r, gamma, sigma, t, TB)
    Z_TO = zero_hw(r, gamma, sigma, t, TO)
    Sz_1 = Sz(gamma, sigma, TO, TB)
    x = (math.log(Z_TB/Strike*Z_TO)+0.5*(Sz_1**2))/Sz_1
    return x


def d2(r, gamma, sigma, t, TO, TB, Strike):
    c = d1(r, gamma, sigma, t, TO, TB, Strike)-Sz(gamma, sigma, TO, TB)
    return c


# Los d1 y d2 son iguales. No es necesario que estén dentro del loop.
def precio(r, gamma, sigma, t, TO, TB, Strike, posicion):
    if posicion == "call":
        Z_TO = zero_hw(r, gamma, sigma, t, TO)
        Z_TB = zero_hw(r, gamma, sigma, t, TB)
        d_11 = d1(r, gamma, sigma, t, TO, TB, Strike)
        d_22 = d2(r, gamma, sigma, t, TO, TB, Strike)
        price = Z_TO*((Z_TB/Z_TO)*norm.cdf(d_11)-Strike*norm.cdf(d_22))
        return price
    if posicion == "put":
        Z_TO = zero_hw(r, gamma, sigma, t, TO)
        Z_TB = zero_hw(r, gamma, sigma, t, TB)
        d_1 = d1(r, gamma, sigma, t, TO, TB, Strike)
        d_2 = d2(r, gamma, sigma, t, TO, TB, Strike)
        price = Z_TO*(Strike*norm.cdf(-d_2)-(Z_TB/Z_TO)*norm.cdf(-d_1))
        return price


def sim_hw_many(gamma, sigma, r0, num_sim, num_steps, seed):

    dt = 1 / 264.0
    num_steps += 1

    alea = np.zeros((num_sim, num_steps))
    rnd.seed(seed)

    for i in range(0, num_sim):
        for j in range(0, num_steps):
            alea[i][j] = rnd.normal()

    theta_array = np.zeros(num_steps)
    tiempo = np.zeros(num_steps)
    for i in range(1, num_steps):
        tiempo[i] = i * dt
        theta_array[i] = theta(i*dt, gamma, sigma)

    sqdt_sigma = math.sqrt(dt) * sigma
    gamma_dt = gamma * dt
    sim = np.zeros((num_sim, num_steps))
    for i in range(0, num_sim):
        sim[i][0] = r0
        r = r0
        for j in range(1, num_steps):
            r = r + theta_array[j - 1] * dt - \
                gamma_dt * r + sqdt_sigma * alea[i][j - 1]
            sim[i][j] = r
    return sim


# ingresamos la funcion zero_hw dentro de precios
def simulaciones_precios(gamma, sigma, t, T, r0, r, num_sim, num_steps, seed):
    precios_matriz = []
    r_matriz = []
    g = sim_hw_many(gamma, sigma, r0, num_sim, num_steps, seed)
    for i in range(num_sim):
        r = g[i][-1]
        r_matriz.append(r)
        precios_matriz.append(zero_hw(r, gamma, sigma, t, T))
    return precios_matriz, r_matriz

In [5]:
# FUNCIÓN QUE CALCULA PRECIO DE UNA OPCIÓN MEDIANTE SIMULACION DE MONTECARLO

# definición de parámetros
gamma = 0.5
sigma = 0.015
t = 0
T_opcion = 1  # Expiración opción = 1
T_bono = 2  # Expiración bono = 2
num_simulaciones = 1000
# Al no usar medida forward, es necesario calcular los factores de dscto a 263 pasos.
num_steps = 263
strike = 1
r_0 = zcurva(0)
semilla = 1234


# Función de Simulación de MC para precios de opciones

# IMPORTANTE MENCIONAR QUE AL NO UTILIZAR MEDIDA FORWARD, DESCONTAMOS LOS PAYOFF A VALOR PRESENTE CON EL FACTOR DE
# DESCUENTO SIMULADO A 263 PASOS. PARA ESTO FUIMOS MULTIPLICANDO TODOS LOS FACTORES POR CADA SIMULACIÓN, PARA LUEGO
# APLICAR EL RESULTADO DE DICHA MULTIPLICACIÓN A CADA PAYOFF SIMULADO Y LUEGO CALCULAR LOS PROMEDIOS.

def Precio_MC(gamma, sigma, t, T, r, r0, num_sim, num_steps, seed, opcion, strike):

    # precios y tasas bajo la misma simulación
    precios_sim, tasas = simulaciones_precios(
        gamma, sigma, t, T, r, r0, num_sim, num_steps, seed)
    dfs = sim_hw_many(gamma, sigma, r0, num_sim, num_steps, seed)
    dt = 1/num_steps
    df = 0

    if opcion == "call":
        payoff_cal = []
        for i in range(len(precios_sim)):
            factores = []
            for j in range(len(dfs[1, :])):
                factores.append(math.exp(dfs[i, j] * (-dt)))
                df_sim = np.prod(factores)
            payoff_cal.append(max(precios_sim[i]-strike, 0) * df_sim)
        precio_opc = np.mean(payoff_cal)

    if opcion == "put":
        payoff_cal_1 = []
        for i in range(len(precios_sim)):
            factores_1 = []
            for j in range(len(dfs[1, :])):
                factores_1.append(math.exp(dfs[i, j] * (-dt)))
                df_sim_1 = np.prod(factores_1)
            payoff_cal_1.append(max(strike-precios_sim[i], 0) * df_sim_1)
        precio_opc = np.mean(payoff_cal_1)
    return precio_opc


# Aplicación de la función con los parámetros escogidos.

call_MC = Precio_MC(gamma, sigma, T_opcion, T_bono, r_0, r_0,
                    num_simulaciones, num_steps, semilla, "call", strike)
call_formula = precio(r_0, gamma, sigma, t, T_opcion, T_bono, strike, "call")


put_MC = Precio_MC(gamma, sigma, T_opcion, T_bono, r_0, r_0,
                   num_simulaciones, num_steps, semilla, "put", strike)
put_formula = precio(r_0, gamma, sigma, t, T_opcion, T_bono, strike, "put")


print(
    f' Precio opción Call con Strike = 1, simulación de montecarlo :  {call_MC}')
print(
    f' Precio opción Call con Strike = 1, fórmulas cerradas de Hull-White:  {call_formula}')
print(f' Precio opción Put con Strike = 1, simulación de montecarlo: {put_MC}')
print(
    f' Precio opción Put con Strike = 1, fórmulas cerradas de Hull-White: {put_formula}')

 Precio opción Call con Strike = 1, simulación de montecarlo :  0.003482213857038766
 Precio opción Call con Strike = 1, fórmulas cerradas de Hull-White:  0.0034869542306079425
 Precio opción Put con Strike = 1, simulación de montecarlo: 0.0036379059587726493
 Precio opción Put con Strike = 1, fórmulas cerradas de Hull-White: 0.0039193358072319925


**Test**

In [6]:
import hull_white as hw

In [7]:
ad_call = hw.zcb_call_put(
    hw.CallPut.CALL,
    strike=strike,
    to=T_opcion,
    tb=T_bono,
    r0=r_0,
    zo=math.exp(-zcurva(T_opcion) * T_opcion),
    zb=math.exp(-zcurva(T_bono) * T_bono),
    gamma=gamma,
    sigma=sigma)
print(strike, ad_call)

1 0.0035283883401179272


In [8]:
ad_put = hw.zcb_call_put(
    hw.CallPut.PUT,
    strike=strike,
    to=T_opcion,
    tb=T_bono,
    r0=r_0,
    zo=math.exp(-zrate(T_opcion) * T_opcion),
    zb=math.exp(-zrate(T_bono) * T_bono),
    gamma=gamma,
    sigma=sigma)
print(strike, ad_put)

1 0.003960769916741769


**AD:** Muy claro el proceso. Hicieron una función. Ojalá hubieran testeado más strikes y/o plazos del bono cupón cero.

In [9]:
## CÁLCULO PRECIO DE UN BONO A 1Y


r=zcurva(1/264) #Tasa instantánea del primer paso desde el que comenzamos a simular.

# Función que estima precio de bono mediante MC.
def Bono_MC(gamma,sigma,r,num_sim,num_steps,seed):
    df = 0
    dt = 1/num_steps
    s = sim_hw_many(gamma,sigma,r,num_sim,num_steps,seed)
    
    for sim in s:
        df += math.exp(-dt * np.sum(sim))
    
    ez = df / num_sim
    return ez


simul_bono = Bono_MC(gamma,sigma,r,num_simulaciones,num_steps,semilla)
precio_mercado = curva['df'].iloc[15] #.iloc 15 se refiere al factor de descuento empírico a 1 año.
error_estimacion = (simul_bono-precio_mercado)/precio_mercado #Error porcentual de la estimación.


print(f'Precio bono cero cupón a 1Y con simulación de Montecarlo: {simul_bono}')
print(f'Precio bono cero cupón a 1Y de mercado: {precio_mercado}')
print(f'Error de la estimación en porcentaje :{error_estimacion: .4%}')


Precio bono cero cupón a 1Y con simulación de Montecarlo: 0.9995669785453031
Precio bono cero cupón a 1Y de mercado: 0.9992978688293875
Error de la estimación en porcentaje : 0.0269%


**AD:** ídem arriba. En este caso muestran el error relativo.